In [1]:
import shapely
import cv2
import numpy as np
import math
import imutils
import fiona
import sys
sys.path.append(r'D:\Code\StreetView\gsv_pano')

from pano import GSV_pano
from PIL import Image
import os
from label_centerlines import get_centerline
from shapely.geometry import Point, Polygon, mapping, LineString, MultiLineString

ModuleNotFoundError: No module named 'cv2'

In [4]:
Polygon([[0, 0], [2, 2], [2, 0]])

ValueError: GEOSGeom_createLinearRing_r returned a NULL pointer

In [19]:
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
    '''
    rows, cols = np.where(x == 1) # .T sets Fortran order down-then-right
    run_lengths = []
    run_rows = []
    prev = -2
    for idx, b in enumerate(cols):
        if (b != prev+1):
            run_lengths.extend((b, 0))
            run_rows.extend((rows[idx],))
        # else:  #(b < prev),  new line
        #     pass
        run_lengths[-1] += 1
        prev = b
    return run_lengths, run_rows
    
def cal_witdh():
    # img_path = r'AZK1jDGIZC1zmuooSZCzEg_DOM.tif'
    # # img_path = r'Ld-CMATy8ZxKap6VAtZTEg_DOM.tif'
    # # img_path = r'v-VR9FB7kCxU1eDLEtFiJQ_DOM.tif'
    # img_path = r'-0D29S37SnmRq9Dju9hkqQ_DOM.tif'
    img_path = r'ZXyk9lKhL5siKJglQPqfMA_DOM_0.05.tif'

    # img_path = r'H:\USC_OneDrive\OneDrive - University of South Carolina\Research\sidewalk_wheelchair\DC_DOMs\-ft2bZI1Ial4C6N_iwmmvw_DOM_0.05.tif'


    panoId_2019 = r'ZXyk9lKhL5siKJglQPqfMA'
    pano1 = GSV_pano(panoId=panoId_2019, crs_local=6847, saved_path=os.getcwd())
    # print(pano1.jdata)
    pano_yaw_deg = pano1.jdata['Projection']['pano_yaw_deg']

    target_ids = [244]
    im_cv = cv2.imread(img_path)

    red_channel = im_cv[:, :, 2]
    class_idx = red_channel
    AOI = np.zeros((len(class_idx), len(class_idx)))
    for i in target_ids:
        AOI = np.logical_or(AOI, class_idx == i)
        # print(AOI)
    # AOI = np.where(AOI == 0, 0, 1).astype(np.uint8)
    # print(AOI)

    morph_kernel_open  = (5, 5)
    morph_kernel_close = (10, 10)
    g_close = cv2.getStructuringElement(cv2.MORPH_RECT, morph_kernel_close)
    g_open  = cv2.getStructuringElement(cv2.MORPH_RECT, morph_kernel_open)

    raw_AOI = np.where(AOI == 0, 0, 1).astype(np.uint8)

    # cv2.imshow("Raw image", np.where(raw_AOI == 0, 0, 255).astype(np.uint8) )

    # yaw_deg =  226.4377593994141 - 90
    # yaw_deg =  92.53645324707031
    yaw_deg =  -pano_yaw_deg
    print("yaw_deg:", yaw_deg)

    cv2_closed = cv2.morphologyEx(raw_AOI, cv2.MORPH_CLOSE, g_close) # fill small gaps
    cv2_opened = cv2.morphologyEx(cv2_closed, cv2.MORPH_OPEN, g_open)

    cv2_opened = np.where(cv2_opened == 0, 0, 255).astype(np.uint8)

    opened_color = cv2.merge((cv2_opened, cv2_opened, cv2_opened))
    img_rotated = imutils.rotate_bound(cv2_opened, yaw_deg)

    # draw lines
    line_cnt = 160
    img_h, img_w = img_rotated.shape
    start_x = 0
    end_x = img_w -1
    interval = int(img_h / line_cnt)
    line_ys = range(interval, img_h, interval)
    line_thickness = 1


    to_RLE = img_rotated[line_ys]
    run_lengths, run_rows = rle_encoding(to_RLE)
    # print("rung_lengths, rows:\n", run_lengths[::2], "\n", run_rows, "\n", run_lengths[1::2])

    AOI = np.where(AOI == 0, 0, 255).astype(np.uint8)

    # cv2.imshow("Raw image", AOI.astype(np.uint8))

    # cv2_closed = np.where(cv2_closed == 0, 0, 255).astype(np.uint8)
    # cv2.imshow("cv2_closed", cv2_closed.astype(np.uint8))

    # for y in line_ys:
    #     cv2.line(opened_color, (start_x, y), (end_x, y), (0, 255, 0), thickness=line_thickness)

    angle_deg = 0
    angle_rad = math.radians(angle_deg)
    lengths = run_lengths[1::2]
    lengths = np.array(lengths)
    new_lengths = lengths.copy()
    new_run_cols = run_lengths[::2].copy()
    pen_lengths = lengths * math.cos(angle_rad)
    to_x = (pen_lengths * math.cos(angle_rad)).astype(int)
    to_y = (pen_lengths * math.sin(angle_rad)).astype(int)
    max_width_meter = 30
    pix_resolution = 0.05
    max_width_pix = int(max_width_meter / pix_resolution)
    for idx, col in enumerate(run_lengths):
        if idx % 2 == 0:
            idx2 = int(idx / 2)
            row = run_rows[idx2] * interval + interval - 1
            radius = 5
            # print(row, col)
            length = run_lengths[idx + 1]
            new_run_cols[idx2] = col
            if (length > max_width_pix) and (idx2 > 0):
                length = new_lengths[idx2 - 1]
                new_run_cols[idx2] = new_run_cols[idx2 - 1]
                print("long length!")
                # print("length, new_lengths[idx2], max_width_pix:", length, new_lengths[idx2], max_width_pix)

            new_lengths[idx2] = length
            new_run_cols[idx2] = new_run_cols[idx2]

            # print("col, new_run_cols[idx2]:", col, new_run_cols[idx2])
            col = new_run_cols[idx2]
            to_x[idx2] = new_lengths[idx2]
            # to_y[idx2] = row

            end_x = col + to_x[idx2]
            end_y = row + to_y[idx2]

            # cv2.line(opened_color, (col, row), (end_x, end_y), (0, 0, 255), thickness=line_thickness)
            # cv2.circle(opened_color, (col, row), radius, (0, 255, 0), line_thickness)
            
            
    # cv2.imshow("cv2_opened", opened_color)

    # find contour
    raw_contours, hierarchy = cv2.findContours(img_rotated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    contours = [np.squeeze(cont) for cont in raw_contours]
    centerlines = get_polygon_centline(contours)

    # all_pair_list = seg_contours(raw_contours[5:6], opened_color)
    img_rotated_color = cv2.merge((img_rotated, img_rotated, img_rotated))
    all_pair_list = seg_contours(raw_contours[:], img_rotated_color)
    end_points = np.zeros((len(all_pair_list) * 2, 2))
    for idx, pair in enumerate(all_pair_list):
        x = int(pair[1])
        y = int(pair[2])
        end_x = int(pair[3])
        end_y = int(pair[4])
        cv2.line(img_rotated_color, (x, y), (end_x, end_y), (255, 0, 0), thickness=line_thickness)
        end_points[idx * 2] = np.array([x, y])
        end_points[idx * 2 + 1] = np.array([end_x, end_y])

    # cv2.imshow("img_rotated added pairs", img_rotated.astype(np.uint8))

    end_points = np.hstack((end_points, np.ones((end_points.shape[0], 1))))
    tx = img_rotated.shape[0] / 2
    ty = img_rotated.shape[1] / 2
    # print("tx, ty:", tx, ty)
    end_points_transed = points_2D_translation(end_points, tx, ty)
    # print("end_points_transed:", end_points_transed[0])

    tx = raw_AOI.shape[0] / 2
    ty = raw_AOI.shape[1] / 2
    #
    # print("tx, ty:", tx, ty)

    end_points_rotated = points_2D_rotated(end_points_transed, yaw_deg)
    # print("end_points_rotated:", end_points_rotated[0])
    end_points_transed = points_2D_translation(end_points_rotated, -tx, ty)

    print("final end_points_transed:", end_points_transed.astype(int))
    line_thickness = 1
    radius = 2
    raw_AOI_color = np.where(raw_AOI == 0, 0, 255).astype(np.uint8)
    raw_AOI_color = cv2.merge((raw_AOI_color, raw_AOI_color, raw_AOI_color))
    line_cnt = len(end_points_transed)
    line_cnt = int(line_cnt)
    end_points_transed = end_points_transed.astype(int)

    dom_path = r'AZK1jDGIZC1zmuooSZCzEg.tif'
    dom_path = r'-ft2bZI1Ial4C6N_iwmmvw_DOM_0.05.tif'
    # im_dom = cv2.imread(dom_path)
    for idx in range(0, line_cnt, 2):
        col = end_points_transed[idx][0]
        row = end_points_transed[idx][1]
        # to_y[idx2] = row

        end_x = end_points_transed[idx + 1][0]
        end_y = end_points_transed[idx + 1][1]
        cv2.line(opened_color, (col, row), (end_x , end_y), (0, 0, 255), thickness=line_thickness)
        # cv2.line(im_dom,       (col, row), (end_x , end_y), (0, 0, 255), thickness=line_thickness)
        # cv2.circle(raw_AOI_color, (end_x, end_y), radius, (0, 255, 0), line_thickness)
        # cv2.circle(raw_AOI_color, (col, row), radius, (0, 255, 0), line_thickness)
    #cv2.imshow("opened_color", opened_color)
    # cv2.imshow("im_dom", im_dom)
    #cv2.imshow("img_rotated_color", img_rotated_color)

    # end_points_transed = end_points_transed[:, 0:2]


    # rotated = imutils.rotate_bound(opened_color, -45)....


    # cv2.imshow("rotated", rotated)
    # to_RLE = np.where(to_RLE == 0, 0, 255).astype(np.uint8)
    # cv2.imshow("to_RLE", to_RLE.astype(np.uint8))

    #cv2.waitKey(0)
    #cv2.destroyAllWindows()


def get_polygon_centline(contours, world_coords=[], segmentize_maxlen=8, max_points=3000, simplification=0.05, smooth_sigma=5):
    if not isinstance(contours, list):
        contours = [contours]
    results = []
    for contour in contours:
        try:
            polygon = Polygon(contour)
            # print(polygon)
            centerlines = get_centerline(polygon, segmentize_maxlen=segmentize_maxlen, max_points=max_points, simplification=simplification, smooth_sigma=smooth_sigma)
            # print(centerline1)
            results.append(centerlines)
        except Exception as e:
            print("Error in get_polygon_centline():", e)
            results.append(0)
            continue

    if len(world_coords) > 0:
        # coords =
        # centerlines = [np.c_[cont, np.ones((len(cont),))] for cont in contours]
        results = [shapely.affinity.affine_transform(cont, world_coords) for cont in results]
        print(results)

    return results


In [20]:
cal_witdh()

yaw_deg: -10.2468376159668


NameError: name 'seg_contours' is not defined

In [22]:
import cv2
import numpy as np
  
# path
path = r'ZXyk9lKhL5siKJglQPqfMA_DOM_0.05.tif'
  
# Reading an image in default
# mode
image = cv2.imread(path)
  
# Window name in which image is
# displayed
window_name = 'Image'
  
# Polygon corner points coordinates
pts = np.array([[25, 70], [25, 160], 
                [110, 200], [200, 160], 
                [200, 70], [110, 20]],
               np.int32)
  
pts = pts.reshape((-1, 1, 2))
  
isClosed = True
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.polylines() method
# Draw a Blue polygon with 
# thickness of 1 px
image = cv2.polylines(image, [pts], 
                      isClosed, color, thickness)
  
# Displaying the image
while(1):
      
    cv2.imshow('image', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break
          
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [13]:
import geopandas as gpd

shp_file = r'H:\USC_OneDrive\OneDrive - University of South Carolina\Research\sidewalk_wheelchair\vectors\panoramas_CRS6487_20m_envelope.shp'
a = r'gdal_translate -projwin 406935.7972999995 135400.57989997734 406975.84730000043 135360.52989997427 -of GTiff "E:/USC_OneDrive/OneDrive - University of South Carolina/Research/sidewalk_wheelchair/DC_sidewalk_raster/sidwalk_dc.img" H:/OUTPUT.tif'

gdf = gpd.read_file(shp_file)

DriverError: 'H:\USC_OneDrive\OneDrive - University of South Carolina\Research\sidewalk_wheelchair\vectors\panoramas_CRS6487_20m_envelope.shp' not recognized as a supported file format.